In [1]:
import pandas as pd
from tqdm import trange, tqdm
import numpy as np
import torch
import sys
from collections import defaultdict
sys.path.append('/mnt/nfs/zhangtl/utils/')
from util import myout
import pickle as pkl

import dgl

Using backend: pytorch


## load

In [2]:
papers = pd.read_csv('../../raw_data/reddit/soc-redditHyperlinks-body.tsv', sep='\t')
# papers.columns = ['source', 'target', 'rating', 'time']
papers = papers.sort_values(by=['TIMESTAMP', 'SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT'])
papers

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES
0,leagueoflegends,teamredditteams,1u4nrps,2013-12-31 16:39:58,1,"345.0,298.0,0.75652173913,0.0173913043478,0.08..."
3,nfl,cfb,1u4sjvs,2013-12-31 17:37:55,1,"1124.0,949.0,0.772241992883,0.0017793594306,0...."
1,theredlion,soccer,1u4qkd,2013-12-31 18:18:37,-1,"101.0,98.0,0.742574257426,0.019801980198,0.049..."
5,dogemarket,dogecoin,1u4w7bs,2013-12-31 18:35:44,1,"1328.0,1110.0,0.768825301205,0.0143072289157,0..."
10,gfycat,india,1u5df2s,2013-12-31 22:27:50,1,"2849.0,2467.0,0.704106704107,0.00737100737101,..."
...,...,...,...,...,...,...
286559,electricskateboards,askreddit,68ipb2s,2017-04-30 16:41:53,1,"1876.0,1567.0,0.78144989339,0.00692963752665,0..."
286539,dndbehindthescreen,dndhomebrew,68i9cxs,2017-04-30 16:43:37,1,"10367.0,8693.0,0.731648500048,0.032796373107,0..."
286560,mgtow,dataisbeautiful,68ipeos,2017-04-30 16:52:59,1,"1129.0,975.0,0.795394154119,0.0150575730735,0...."
286558,mmorpg,blackdesertonline,68ip5os,2017-04-30 16:54:08,1,"1100.0,909.0,0.778181818182,0.00181818181818,0..."


In [12]:
a = '2013-12-31 18:35:44'
int(''.join(a.split(' ')[0].split('-')))

20131231

In [3]:
papers['time'] = papers['TIMESTAMP'].apply(lambda u: int(''.join(u.split(' ')[0].split('-'))))
papers = papers.sort_values(by=['time', 'SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT'])
papers

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES,time
5,dogemarket,dogecoin,1u4w7bs,2013-12-31 18:35:44,1,"1328.0,1110.0,0.768825301205,0.0143072289157,0...",20131231
10,gfycat,india,1u5df2s,2013-12-31 22:27:50,1,"2849.0,2467.0,0.704106704107,0.00737100737101,...",20131231
0,leagueoflegends,teamredditteams,1u4nrps,2013-12-31 16:39:58,1,"345.0,298.0,0.75652173913,0.0173913043478,0.08...",20131231
3,nfl,cfb,1u4sjvs,2013-12-31 17:37:55,1,"1124.0,949.0,0.772241992883,0.0017793594306,0....",20131231
8,posthardcore,bestof2013,1u5ccus,2013-12-31 23:16:20,1,"3927.0,3488.0,0.719887955182,0.0229182582124,0...",20131231
...,...,...,...,...,...,...,...
286371,vegancirclejerk,vegan,68ei3os,2017-04-30 01:06:17,1,"99.0,98.0,0.737373737374,0.040404040404,0.0404...",20170430
286515,videoediting,videos,68hrbrs,2017-04-30 13:12:57,1,"525.0,454.0,0.765714285714,0.0152380952381,0.0...",20170430
286553,weedstocks,wallstreetbets,68ihozs,2017-04-30 14:28:25,1,"870.0,748.0,0.766666666667,0.00459770114943,0....",20170430
286462,wmaf,hapas,68gyun,2017-04-30 06:57:04,-1,"1171.0,995.0,0.794192997438,0.00597779675491,0...",20170430


In [4]:
myout(papers['time'].unique())

 : shape=(1217,), [20131231 20140101 20140102 ... 20170428 20170429 20170430]


## cut

In [27]:
len(papers) / 20

14328.05

In [5]:
cut_step = len(papers) / 20
papers['new_time'] = (papers.index.values / cut_step).astype(np.int)
papers = papers.sort_values(by=['new_time', 'SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT'])
papers

/tmp/ipykernel_3955944/3770066049.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  papers['new_time'] = (papers.index.values / cut_step).astype(np.int)


,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES,time,new_time
631,100movies365days,movieaweek,1ul33ms,2014-01-06 19:38:09,1,"2294.0,1927.0,0.755013077594,0.0130775937228,0...",20140106,0
1820,100movies365days,movieaweek,1v90lzs,2014-01-14 18:59:17,1,"1553.0,1322.0,0.743077913715,0.0180296200901,0...",20140114,0
13022,18bfriendzonest,adviceanimals,21djp2,2014-03-25 18:49:17,-1,"2436.0,2153.0,0.649014778325,0.00697865353038,...",20140325,0
13044,18bfriendzonest,askwomen,21dqe7s,2014-03-25 21:24:40,1,"1394.0,1271.0,0.766857962697,0.0243902439024,0...",20140325,0
13046,18bfriendzonest,redditgetsdrawn,21dqe7s,2014-03-25 21:24:40,1,"1394.0,1271.0,0.766857962697,0.0243902439024,0...",20140325,0
...,...,...,...,...,...,...,...,...
281201,zootopia,place,654e4xs,2017-04-13 03:33:56,1,"1125.0,995.0,0.759111111111,0.016,0.0186666666...",20170413,19
281200,zootopia,thefinalclean,654e4xs,2017-04-13 03:33:56,1,"1125.0,995.0,0.759111111111,0.016,0.0186666666...",20170413,19
281447,zootopia,whowouldwin,65aieis,2017-04-13 20:35:49,1,"421.0,366.0,0.771971496437,0.0142517814727,0.0...",20170413,19
284554,zotac,nvidia,679cp3s,2017-04-24 09:27:05,1,"328.0,288.0,0.75,0.0213414634146,0.03658536585...",20170424,19


In [7]:
for ii in range(0, 20):
    print(ii, len(np.where(papers['new_time'].to_numpy()==ii)[0]))

0 14329
1 14328
2 14328
3 14328
4 14328
5 14328
6 14328
7 14328
8 14328
9 14328
10 14328
11 14328
12 14328
13 14328
14 14328
15 14328
16 14328
17 14328
18 14328
19 14328


## build graph

In [8]:
feat = [float(item) for item in papers['PROPERTIES'].iloc[0].split(',')]
myout(feat)

feat : len=86, list([2294.0, 1927.0, 0.755013077594, ..., 0.004914004914004914, 0.0, 0.0])


In [9]:
def update_idx(idx, dic, cnt, feats, feat_dim, no_emb, ff):
    if idx not in dic:
        dic[idx] = cnt
        cnt += 1
        feats.append(torch.tensor(ff).to(torch.float32))
    return dic, cnt, feats, no_emb

In [13]:
start_year, end_year = 0, 20
feat_dim = 128
index = {}

id2nid, cnt, no_emb = {}, 0, 0
lst, feats = [], []
last_len, last_year = 0, start_year

for ii in trange(len(papers)):
    year = int(papers['new_time'].iloc[ii])
    if year != last_year: # 进入下一个年份，则更新上个年份
        index[last_year] = [len(feats), last_len, len(lst)] # 记录当前年份的节点数、eind的起始和结束
        last_len = len(lst)
        last_year = year

    source = str(papers['SOURCE_SUBREDDIT'].iloc[ii])
    target = str(papers['TARGET_SUBREDDIT'].iloc[ii])
    weight = int(papers['LINK_SENTIMENT'].iloc[ii])
    ff = [float(item) for item in papers['PROPERTIES'].iloc[ii].split(',')]
    
    id2nid, cnt, feats, no_emb = update_idx(source, id2nid, cnt, feats, feat_dim, no_emb, ff)
    id2nid, cnt, feats, no_emb = update_idx(target, id2nid, cnt, feats, feat_dim, no_emb, ff)
    
    lst.append((id2nid[source], id2nid[target], weight))

index[year] = [len(feats), last_len, len(lst)]
feat = torch.stack(feats)
src = torch.tensor([item[0] for item in lst])
tgt = torch.tensor([item[1] for item in lst])
rel = torch.tensor([item[2] for item in lst])
eind = torch.stack([src, tgt])

flag1, flag2 = '' in id2nid, np.nan in id2nid
mm = torch.max(eind).item()
print(f'feat:{feat.shape}, eind:{eind.shape},{mm}, rel:{len(rel)}, {flag1},{flag2}, {no_emb}')
inputs = {'feat': feat, 'eind': eind, 'rel': rel, 'id2nid': id2nid, 'index': index}
myout(feat, eind, rel, id2nid)

100%|██████████| 286561/286561 [00:15<00:00, 18119.24it/s]


feat:torch.Size([35776, 86]), eind:torch.Size([2, 286561]),35775, rel:286561, False,False, 0
feat : shape=torch.Size([35776, 86])
tensor([[2.2940e+03, 1.9270e+03, 7.5501e-01,  ..., 4.9140e-03, 0.0000e+00,
         0.0000e+00],
        [2.2940e+03, 1.9270e+03, 7.5501e-01,  ..., 4.9140e-03, 0.0000e+00,
         0.0000e+00],
        [2.4360e+03, 2.1530e+03, 6.4901e-01,  ..., 5.4795e-03, 0.0000e+00,
         5.4795e-03],
        ...,
        [1.4007e+04, 1.2086e+04, 6.8837e-01,  ..., 4.2753e-04, 4.2753e-04,
         8.5507e-04],
        [1.4800e+02, 1.3600e+02, 7.4324e-01,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [3.2800e+02, 2.8800e+02, 7.5000e-01,  ..., 0.0000e+00, 1.9231e-02,
         0.0000e+00]])
eind : shape=torch.Size([2, 286561])
tensor([[    0,     0,     2,  ..., 19419, 35775,  7327],
        [    1,     1,     3,  ...,   142,  1240,  6169]])
rel : shape=torch.Size([286561]), tensor([ 1,  1, -1,  ...,  1,  1,  1])
id2nid : len=35776, dict(ovies365days: 0, movie

In [14]:
index

{0: [5029, 0, 14329],
 1: [7670, 14329, 28657],
 2: [9840, 28657, 42985],
 3: [11825, 42985, 57313],
 4: [13560, 57313, 71641],
 5: [15370, 71641, 85969],
 6: [17090, 85969, 100297],
 7: [18810, 100297, 114625],
 8: [20522, 114625, 128953],
 9: [22112, 128953, 143281],
 10: [23537, 143281, 157609],
 11: [25024, 157609, 171937],
 12: [26525, 171937, 186265],
 13: [27940, 186265, 200593],
 14: [29453, 200593, 214921],
 15: [30791, 214921, 229249],
 16: [32116, 229249, 243577],
 17: [33386, 243577, 257905],
 18: [34506, 257905, 272233],
 19: [35776, 272233, 286561]}

In [16]:
pkl.dump(inputs, open(f'../../../01_process/data_papers/rdb/inputs.pkl', 'wb'))

## labels

In [18]:
start_year, end_year = 0, 20
cites = {}
for year in range(start_year, end_year):
    cites[year] = defaultdict(int)

for ii in trange(len(papers)):
    year = int(papers['new_time'].iloc[ii])
    target = str(papers['TARGET_SUBREDDIT'].iloc[ii])
    cites[year][target] += 1
myout(cites[5])

100%|██████████| 286561/286561 [00:03<00:00, 77336.19it/s]

 : len=3486, dict([keto: 10, writingprompts: 123, askreddit: 346, soccer: 63, runescape: 4, fantheories: 8, ...])


In [22]:
labels = {}
id2nid, index = inputs['id2nid'], inputs['index']
nid2id = {v:k for k,v in id2nid.items()}
for year in range(start_year, end_year):
    left, right = index[year][1:]
    eind = inputs['eind'][:, left:right]
    
    nids = torch.unique(eind[0, :]).tolist()
    ids = [nid2id[nid] for nid in nids]
    
    pdf = pd.DataFrame({'id': ids, 'nid': nids})
    tbar = trange(year+1, end_year, desc=str(year))
    for yy in tbar:
        cdf = pd.DataFrame({'id': list(cites[yy].keys()), str(yy): list(cites[yy].values())})
        cdf[str(yy)] = cdf[str(yy)].astype('float32')
        
        pdf = pd.merge(pdf, cdf, how='left', on='id')
        tbar.set_postfix(year=year, pdf=len(pdf))
    pdf.fillna(0, inplace=True)
    labels[year] = pdf

print(len(labels))

18: 100%|██████████| 1/1 [00:00<00:00, 72.16it/s, pdf=4289, year=18]
19: 0it [00:00, ?it/s]

20


In [23]:
labels[5]

,id,nid,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,askwomen,4,6.0,11.0,11.0,8.0,11.0,4.0,14.0,5.0,11.0,6.0,6.0,14.0,10.0,5.0
1,redditgetsdrawn,5,1.0,3.0,1.0,6.0,2.0,2.0,3.0,3.0,7.0,5.0,4.0,6.0,2.0,2.0
2,askreddit,9,425.0,364.0,338.0,341.0,389.0,287.0,292.0,309.0,349.0,355.0,294.0,304.0,248.0,261.0
3,dogecoin,11,20.0,5.0,59.0,3.0,6.0,3.0,14.0,2.0,2.0,3.0,4.0,4.0,2.0,1.0
4,2007scape,12,8.0,3.0,9.0,4.0,5.0,13.0,14.0,10.0,17.0,11.0,6.0,11.0,13.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4324,zengmfootball,15363,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4325,ziincworks,15365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4326,zmarter,15366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4327,zombicide,15368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
pkl.dump(labels, open(f'../../../01_process/data_papers/rdb/labels.pkl', 'wb'))

In [25]:
def cumulative_log(df):
    colsn = list(df.columns)
    for i in range(3, len(colsn)):
        df[colsn[i]] += df[colsn[i-1]]
    df.iloc[:, 2:] = np.log(df.iloc[:, 2:] + 1)
    return df

labels_cum_log = {}
for year in range(start_year, end_year-1):
    labels_cum_log[year] = cumulative_log(labels[year])
labels_cum_log[end_year-2] = labels[end_year-2]
print(len(labels_cum_log))
labels_cum_log[5]

19


,id,nid,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,askwomen,4,1.945910,2.890372,3.367296,3.610918,3.871201,3.951244,4.189655,4.262680,4.406719,4.477337,4.543295,4.682131,4.770685,4.812184
1,redditgetsdrawn,5,0.693147,1.609438,1.791759,2.484907,2.639057,2.772589,2.944439,3.091043,3.367296,3.526361,3.637586,3.784190,3.828641,3.871201
2,askreddit,9,6.054440,6.672033,7.028202,7.292337,7.527256,7.670895,7.798523,7.917901,8.037543,8.146130,8.227910,8.305978,8.365439,8.424419
3,dogecoin,11,3.044523,3.258096,4.442651,4.477337,4.543295,4.574711,4.709530,4.727388,4.744932,4.770685,4.804021,4.836282,4.852030,4.859812
4,2007scape,12,2.197225,2.484907,3.044523,3.218876,3.401197,3.761200,4.043051,4.204693,4.430817,4.553877,4.615120,4.718499,4.828314,4.912655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4324,zengmfootball,15363,0.000000,0.000000,0.693147,1.098612,1.098612,1.098612,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294
4325,ziincworks,15365,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4326,zmarter,15366,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4327,zombicide,15368,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [26]:
pkl.dump(labels_cum_log, open(f'../../../01_process/data_papers/rdb/labels_cum_log.pkl', 'wb'))